### 참고한 사이트 및 코드

In [ ]:
# https://yamalab.tistory.com/64
req = requests.get('https://music.bugs.co.kr/chart')
html = req.content
soup = BeautifulSoup(html, 'lxml') # pip install lxml
list_song = soup.find_all(name="p", attrs={"class":"title"})
list_artist = soup.find_all(name="p", attrs={"class":"artist"})

# 곡명 추출
for index in range(0, len(list_song)):
    title = list_song[index].find('a').text
    print(index+1, ' : ', title)
    if index == 100:
        break

# 피처링 제거
for index in range(0, len(list_song)):
    title = list_song[index].find('a').text
    print(index+1, ' : ', title.split("(")[0])
    if index == 100:
        break

# csv로 저장
import csv

with open('melon_chart.csv', 'w', encoding='utf-8') as file:
    writer = csv.writer(file, delimiter=',')
    writer.writerow(['rank', 'song', 'artist'])
    for index in range(0, len(list_song)):
        title = list_song[index].find('a').text
        artist = list_artist[index].find('a').text
        writer.writerow([index+1, title, artist])
        if index == 100:
            break

# 저장된 파일 pd로 읽기
import pandas as pd
datas = pd.read_csv('melon_chart.csv')

## bugs crawler

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# -*- coding: utf-8 -*-
import requests
from bs4 import BeautifulSoup
import re

In [ ]:
## 0.start : 벅스 뮤직 PD로 접속
req = requests.get('https://music.bugs.co.kr/musicpd?wl_ref=M_left_02_04')
html = req.content
soup = BeautifulSoup(html, 'lxml') # pip install lxml

In [ ]:
## 1.뮤직스타일 링크 추출 -> dict으로 저장
dict_categorytheme={}
for a_tag in soup.select('ul.listCategoryTheme')[0].select('a.hyrend'):
    dict_categorytheme[a_tag.get_text().rstrip()]=a_tag['href'] #장르 오른쪽 공백 지우기
dict_categorytheme

In [ ]:
## 2. 뮤직스타일 링크로 삽입 
for url_categorytheme in dict_categorytheme.values():
    print('check',url_categorytheme) # : 22개 장르 모두 나오는 것 확인
    req_category = requests.get(url_categorytheme)
    html_category = req_category.content
    soup_category = BeautifulSoup(html_category, 'lxml') # pip install lxml
    #print(soup_category)

### '신나는' 장르에서만 페이지 불러오기

In [ ]:
### 2-1. 뮤직스타일 링크에서 각 페이지 불러오기 : 신나는 장르에서만!!
req_category = requests.get('https://music.bugs.co.kr/musicpd?tag_id=429')
html_category = req_category.content
soup_category = BeautifulSoup(html_category, 'lxml')

### 2-2. page iteration
total_num = soup_category.select_one('p.desc').get_text()
total_num = int(re.sub(r'\D', '', total_num))
page_num = total_num//24 + 2

### 몇 페이지인지
base_url = soup.select('div.paging > a')[0]['href'][:-1] # https://music.bugs.co.kr/musicpd?order=list&page=1
for n_th_page in range(1,page_num):
    url_categorypages=base_url+f'{n_th_page}'
    #print(url_categorypages) #: 1에서 110페이지까지 잘 나옴

### 1페이지에서만 앨범링크 긁어서 '신나는'-1페이지 data 만들기

In [ ]:
# 각 페이지에서 앨범 링크 긁기 : 1번째 페이지에서만!!
'''
1. dict_categoryalbum={}
ex. {'2020 일본 레코드 대상 노미네이트': 'https://music.bugs.co.kr/musicpd/albumview/47364?wl_ref=list_mab_03',
 'UNKNOWN, 나만 알고 싶은 Track #8': 'https://music.bugs.co.kr/musicpd/albumview/47260?wl_ref=list_mab_03',

2. dict_category={} <- dict_categoryalbum 곡들 다 넣어야 됨. 어느 정보까지..?
'''
dict_category={}

########## 빈 칸 채우기 용도
dict_category['category_theme'] = []
dict_category['artist_disp_nm']= []
##dict_category['artist_id']=[] : 처음에 넣었다가 필요없을 것같아서 뺌.
dict_category['track_title']= []
dict_category['album_style']=[]
##########

dict_categoryalbum={}
'''
일단 1페이지만 가져오기
'''
req_pages = requests.get('https://music.bugs.co.kr/musicpd?order=list&page=1')
html_pages = req_pages.content
soup_pages = BeautifulSoup(html_pages, 'lxml')

for album_tag in soup_pages.find_all('a', class_="title hyrend"):
    dict_categoryalbum[album_tag['title']]=album_tag['href']
#print(dict_categoryalbum) # 순서대로는 아닌데 쨋든 album dict 완료 

## 페이지별로 dict 다 돌린 다음에 앨범별로 돌리면 될 듯. 일단 1페이지에 해당하는 것만 돌려봤습니다.
for url_categoryalbum in dict_categoryalbum.values():
    req_album = requests.get(url_categoryalbum)
    html_album = req_album.content
    soup_album = BeautifulSoup(html_album, 'lxml') # pip install lxml
    ####### 앨범 특징 : 팀원 피드백) 곡 제목-스타일-음악에 대표되는 감성? 을 위주로 중 음악에 대표되는 감성을 앨범 #로 태그해놔서 가져와봤어요

    ele_album_style = str() 
    for a_tag in soup_album.select('div.tag > a.hyrend'):
        ele_album_style+=a_tag.get_text()

    for music_tag in soup_album.find_all('a',attrs={'class': 'btnActions','album_title':False}): # soup_album.select('a.btnActions')
        #print(music_tag)
        dict_category['category_theme'].append('신나는')
    # 수작업으로 이렇게 했는데 추후에 dict에서 key 빼서 할 수도 있을 것같아요. 근데 22개 장르라 그냥 수작업으로 해도 괜찮을 것같고 그렇습니다아 category
        dict_category['artist_disp_nm'].append(music_tag['artist_disp_nm'])
     #dict_category['artist_id'].append(music_tag['artist_id'])
        dict_category['track_title'].append(music_tag['track_title'])
        dict_category['album_style'].append(ele_album_style)

In [ ]:
dict_category

In [ ]:
import pandas as pd

pd.DataFrame.from_dict(dict_category)